In [1]:
import math as m
import numpy as np
import pandas as pd
import gspread as gs

# Test Code -  Hopefully works out
1. Need list or Database to take in Measured & Calculated values per droplet ionization
> Droplet Database: 
>> Idetifier: Droplet(i) for $i \in (1,2,3...)$ for a specific ionization<br>

>> Set:<br>
>> - V, b, g, p, $\eta_0$, $\rho$, d<br>

>> Measured/Computed:<br>
>> - $v_{y0i}$ = $\frac{ x_{i0}}{ t_{i0}}$<br>
>> - $a$<br>
>> - $\eta_{eff}$<br>
>> - $v_{yEi}$= $\frac{ x_{iE}}{ t_{iE}}$<br>
>> - $q$<br>


2. Need equation/function for radius (simplified for coding)
> $$a = \sqrt{\left(\frac{b}{2p}\right)^2 + \frac{9v_{y0}\eta_0}{2 \rho g}} -\left(\frac{b}{2p}\right)$$
> > $$a = \sqrt{\left(\alpha\right)^2 + \beta} -\left(\alpha\right)$$


3. Need equation/function for charge (simplified for coding)
> $$ q = \frac{4}{3}\pi \rho g \frac{d}{V}\left(
\sqrt{\left(\frac{b}{2p}\right)^2 + \frac{9v_{y0}\eta_0}{2 \rho g}} -\left(\frac{b}{2p}\right)
\right)^3 \cdot \frac{v_{yE}-v_{y0}}{v_{y0}}$$
> > $$ q = \frac{4}{3}\pi \rho g \frac{d}{V}a^3 \cdot \frac{v_{yE}-v_{y0}}{v_{y0}}$$

4. Need equations for Error propagation 
5. Need Graphs see images saved from speaking with the professor 

# Set constants provided in Manual 

In [2]:
# Constants 

b   = 8.20E-3         # Viscostiy Correction Factor (given)
g   = 9.80            # Gravity (given)
p   = 1.013E5         # Atmospheric Pressure (at room temp? - given in docs)
n_o = 1.824E-5        # Tabulated Viscosity/ Viscocity of air (given)
rho1 = 8.86E2         # Density of oil droplet (given)
rho2 = 0              # To be determined for accuracy   

#Stored List 
Constants = [b,g,p,n_o,rho1,rho2]

# format string with f"{variable:.ne}" where n is the number of decimal values 
# e calls scientific notation

print("Given Constants:".upper())
print(f"Viscostiy Correction Factor:\t\t\t b \t= {b:.2e} N/m")
print(f"Gravity:\t\t\t\t\t g \t= {g:.2e} m/s^2")
print(f"Atmospheric Pressure:\t\t\t\t p \t= {p:.3e} N/m^2")
print(f"Viscocity of air(ucorrected for radius):\t n_o \t= {n_o:.3e} N-s/m^2")
print(f"Densidty of Oil Droplet:\t\t\t rho \t= {rho1:.2e} kg/m^m")


GIVEN CONSTANTS:
Viscostiy Correction Factor:			 b 	= 8.20e-03 N/m
Gravity:					 g 	= 9.80e+00 m/s^2
Atmospheric Pressure:				 p 	= 1.013e+05 N/m^2
Viscocity of air(ucorrected for radius):	 n_o 	= 1.824e-05 N-s/m^2
Densidty of Oil Droplet:			 rho 	= 8.86e+02 kg/m^m


# Measured/ Calculated Values and their associated Error 
- Voltage across plates (V): --------------------------------------- V
- Separation of Plates (m):--------------------------------------- d 
- Droplet downward velocity with no Field (\frac{m}{s}):--------$v_{y0}$
- Droplet radius (m):--------------------------------------------- a 
- Viscosity of air, corrected for radius ($\frac{N-s}{m^2}$):------------------$\eta_{eff}(a)$
- Droplet downward velocity with E Field (\frac{m}{s}):---------$v_{yE}$
- Droplet Total charge (C):--------------------------------------- q 






### Separation (d) and it's error: (?)


In [3]:
# Separation of plates (d) 
d   = 7.752E-3      # meters

# Error in Separation (?)

# Data Frame 

In [4]:
# Constants 
droplet_constants = pd.read_csv('Droplet-Data-Constants.csv')
#droplet_constants

In [5]:
# TestValues
droplets_testvalues = pd.read_csv('Droplet-Data-Test.csv')
#droplets_testvalues

## Functions for $a$ (might need to adjust for array)

> $$a = \sqrt{\left(\frac{b}{2p}\right)^2 + \frac{9v_{y0}\eta_0}{2 \rho g}} -\left(\frac{b}{2p}\right)$$
> > $$a = \sqrt{\left(\alpha\right)^2 + \beta} -\left(\alpha\right)$$


In [6]:
# Fuction for radius a 
# assuming b, p, n_o, g are all set 
# inputs (x,t) for E = 0 

def radius(x,t):     
    # Components of a 
    # terminal velocity with E = 0 
    v_y0 = (x/t)

    # alpha 
    alpha = b/(2*p)

    #beta
    beta = (9*v_y0*n_o)/(2*rho1*g)
    
    #radius a 
    a = np.sqrt((alpha**2)+beta) - alpha
   
    return a


## Function for charge $q$

- $q_1 = q_2=  q$ They are just expressed differently 

> $$ q = \frac{4}{3}\pi \rho g \frac{d}{V}\left(
\sqrt{\left(\frac{b}{2p}\right)^2 + \frac{9v_{y0}\eta_0}{2 \rho g}} -\left(\frac{b}{2p}\right)
\right)^3 \cdot \frac{v_{yE}-v_{y0}}{v_{y0}}$$
> > $$ q_1 = \frac{4}{3}\pi \rho g \frac{d}{V}a^3 \cdot \frac{v_{yE}-v_{y0}}{v_{y0}}$$

>>$$ q_2 = 18 \pi d \left( \frac{\eta_0^3}{2g\rho}\right)^{\frac{1}{2}}\cdot 
\left( \frac{1}{(1+\frac{b}{pa})}\right)^{\frac{3}{2}} \sqrt{v_{y0}}\cdot
\frac{(v_{yE}-v_{y0})}{V}
$$

In [7]:
# q1 

def q1(a, V, x_0, t_0, x_E, t_E):
    #velocity E=0 
    v_y0 = x_0/t_0

    #velocity E != 0
    v_yE = x_E/t_E

    #left term (lt) 
    lt = (4/3)*np.pi*rho1*g*(d/V)*a**3

    #right term (rt)
    rt = (v_yE - v_y0)/ v_y0

    q1 = lt*rt
    
    return q1


# q2

def q2(a, V, x_0, t_0, x_E, t_E):
    #velocity E=0 
    v_y0 = x_0/t_0

    #velocity E != 0
    v_yE = x_E/t_E
    
    #term1
    t1 = 18*np.pi*d*np.sqrt(n_o**3/(2*g*rho1))

    #term2
    t2 =(1/(1+(b/(p*a))))**(3/2)

    #term3
    t3 = np.sqrt(v_y0)* ((v_yE-v_y0)/V)

    q2 = t1*t2*t3

    return q2

## Function for Effective Viscosity $\eta_{eff}(a)$

$$\eta_{eff}(a) = \frac{\eta_0}{1 + \frac{b}{pa}}
$$

In [8]:
# effective viscosity 

def neff(a): 
    n_effa = n_o/(1 + (b/(p*a)))

    return n_effa
    

# Test Charge Values 
- Manual inputs (very very approximate, needs error analysis)
> Voltage V = 501V

> Falling $E=0$ (xf,tf) = (.5E-3m,5s)

> Rising $E\neq 0$ (xr,tr) = (.5E-3, 4s)


In [12]:
# Inputs 
# Fall 
x = .5E-3      # mm
t = 52-47   #s

# Rise
xr = .5E-3
tr= 4

#Calculate Radius
a1 =radius(x,t)
print(f"Radius of Droplet = {a1:.3e}\n")

#Calculate Charge
q = q2(a1,501,x,t, xr,tr,)
print(f"Charge on droplet= {q:.3e}\n")

#Compare to known charge 
q0 = 1.60217E-19

print(f"Fundamental Charge q_0= {q0:.5e} " )



Radius of Droplet = 9.326e-07

Charge on droplet= 1.141e-19

Fundamental Charge q_0= 1.60217e-19 
